<a href="https://colab.research.google.com/github/JoelPasapera/AI-Strategies-in-python-google-colab-/blob/main/Red_neuronal_trading_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# =============================================
# CONFIGURACIÓN INICIAL Y INSTALACIÓN DE LIBRERÍAS
# =============================================
%%capture
!pip install tensorflow #==2.13.0
!pip install keras-tuner
!pip install ta
!pip install plotly
!pip install yfinance
!pip install scikit-optimize

In [15]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [12]:
# =============================================
# CARGA Y PREPROCESAMIENTO DE DATOS
# =============================================

from google.colab import files
uploaded = files.upload()

# Cargar datos
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name, encoding='utf-16')

print(f"Dataset cargado: {df.shape}")
print(df.head())

# Procesamiento de fecha
df['date'] = pd.to_datetime(df['date'], format='%Y.%m.%d %H:%M:%S')
df = df.sort_values('date').reset_index(drop=True)

Saving GBPUSD_16385_minutes.csv to GBPUSD_16385_minutes (2).csv
Dataset cargado: (11684, 5)
                  date     open     high      low    close
0  2025.11.14 23:00:00  1.31677  1.31745  1.31645  1.31744
1  2025.11.14 22:00:00  1.31646  1.31707  1.31614  1.31677
2  2025.11.14 21:00:00  1.31603  1.31676  1.31561  1.31646
3  2025.11.14 20:00:00  1.31608  1.31632  1.31493  1.31603
4  2025.11.14 19:00:00  1.31548  1.31630  1.31507  1.31609


In [13]:
# ============================================
# CONFIGURACIONES GENERALES
# ============================================
# -*- coding: utf-8 -*-
"""Modelo Trading para GBPUSD.ipynb

Automatically generated by Colaboratory.

Original file is located at:
    https://colab.research.google.com/drive/1abc123xyz
"""

# Define cuantas epocas tendrá el modelo para el entrenamiento
EPOCHS: int = 200
BATCH_SIZE: int = 256

# Configuración para máxima performance
tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(True)

# Verificar GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"{len(gpus)} GPUs disponibles")
    except RuntimeError as e:
        print(e)

In [14]:

# =============================================
# ANÁLISIS EXPLORATORIO Y FEATURE ENGINEERING
# =============================================

import ta
from ta import add_all_ta_features
from ta.utils import dropna

class AdvancedFeatureEngineer:
    def __init__(self, df):
        self.df = df.copy()

    def add_price_features(self):
        """Características basadas en precio"""
        df = self.df

        # Returns y volatilidad
        df['returns'] = df['close'].pct_change()
        df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
        df['volatility'] = df['returns'].rolling(window=20).std()
        df['price_range'] = (df['high'] - df['low']) / df['close']
        df['body_size'] = abs(df['close'] - df['open']) / df['close']

        # Medias móviles
        for window in [5, 10, 20, 50]:
            df[f'sma_{window}'] = df['close'].rolling(window=window).mean()
            df[f'ema_{window}'] = df['close'].ewm(span=window).mean()
            df[f'price_vs_sma_{window}'] = df['close'] / df[f'sma_{window}'] - 1

        return df

    def add_technical_indicators(self):
        """Indicadores técnicos avanzados"""
        df = self.df

        # RSI
        df['rsi'] = ta.momentum.RSIIndicator(df['close']).rsi()

        # MACD
        macd = ta.trend.MACD(df['close'])
        df['macd'] = macd.macd()
        df['macd_signal'] = macd.macd_signal()
        df['macd_histogram'] = macd.macd_diff()

        # Bollinger Bands
        bollinger = ta.volatility.BollingerBands(df['close'])
        df['bb_upper'] = bollinger.bollinger_hband()
        df['bb_lower'] = bollinger.bollinger_lband()
        df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['close']
        df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])

        # Ichimoku Cloud
        ichimoku = ta.trend.IchimokuIndicator(df['high'], df['low'])
        df['ichimoku_a'] = ichimoku.ichimoku_a()
        df['ichimoku_b'] = ichimoku.ichimoku_b()
        df['ichimoku_base'] = ichimoku.ichimoku_base_line()
        df['ichimoku_conversion'] = ichimoku.ichimoku_conversion_line()

        # Volumen (aunque no tenemos volumen, usamos rangos de precio como proxy)
        df['atr'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close']).average_true_range()

        # Stochastic
        stoch = ta.momentum.StochasticOscillator(df['high'], df['low'], df['close'])
        df['stoch_k'] = stoch.stoch()
        df['stoch_d'] = stoch.stoch_signal()

        # ADX
        df['adx'] = ta.trend.ADXIndicator(df['high'], df['low'], df['close']).adx()

        return df

    def add_time_features(self):
        """Características temporales"""
        df = self.df

        df['hour'] = df['date'].dt.hour
        df['day_of_week'] = df['date'].dt.dayofweek
        df['day_of_month'] = df['date'].dt.day
        df['month'] = df['date'].dt.month
        df['week_of_year'] = df['date'].dt.isocalendar().week

        # Ciclos temporales
        df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
        df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
        df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
        df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

        return df

    def add_statistical_features(self):
        """Características estadísticas avanzadas"""
        df = self.df

        # Rolling statistics
        for window in [10, 20, 50]:
            df[f'rolling_mean_{window}'] = df['close'].rolling(window).mean()
            df[f'rolling_std_{window}'] = df['close'].rolling(window).std()
            df[f'rolling_skew_{window}'] = df['close'].rolling(window).skew()
            df[f'rolling_kurt_{window}'] = df['close'].rolling(window).kurt()

        # Z-score
            df[f'z_score_{window}'] = (df['close'] - df[f'rolling_mean_{window}']) / df[f'rolling_std_{window}']

        # Hurst Exponent (simplificado)
        def hurst_exponent(ts):
            lags = range(2, 20)
            tau = [np.std(np.subtract(ts[lag:], ts[:-lag])) for lag in lags]
            poly = np.polyfit(np.log(lags), np.log(tau), 1)
            return poly[0]

        df['hurst_20'] = df['close'].rolling(100).apply(hurst_exponent, raw=True)

        return df

    def add_target_variables(self):
        """Variables objetivo para el modelo"""
        df = self.df

        # Retornos futuros (1h, 4h, 24h)
        df['target_1h'] = df['close'].shift(-1) / df['close'] - 1
        df['target_4h'] = df['close'].shift(-4) / df['close'] - 1
        df['target_24h'] = df['close'].shift(-24) / df['close'] - 1

        # Dirección del mercado
        df['target_direction_1h'] = np.where(df['target_1h'] > 0, 1, 0)
        df['target_direction_4h'] = np.where(df['target_4h'] > 0, 1, 0)

        # Volatilidad futura
        df['target_volatility_4h'] = df['returns'].shift(-4).rolling(4).std()

        return df

    def engineer_all_features(self):
        """Ejecutar todo el feature engineering"""
        df = self.add_price_features()
        df = self.add_technical_indicators()
        df = self.add_time_features()
        df = self.add_statistical_features()
        df = self.add_target_variables()

        # Eliminar filas con NaN
        df = df.dropna()

        return df

# Aplicar feature engineering
feature_engineer = AdvancedFeatureEngineer(df)
df_enhanced = feature_engineer.engineer_all_features()

print(f"Dataset después del feature engineering: {df_enhanced.shape}")

# =============================================
# PREPARACIÓN DE DATOS PARA EL MODELO
# =============================================

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split

class DataPreprocessor:
    def __init__(self, df, sequence_length=60, test_size=0.2, val_size=0.1):
        self.df = df
        self.sequence_length = sequence_length
        self.test_size = test_size
        self.val_size = val_size
        self.scalers = {}

    def prepare_features(self):
        """Seleccionar y preparar características"""
        # Excluir columnas no numéricas y targets
        exclude_cols = ['date', 'target_1h', 'target_4h', 'target_24h',
                       'target_direction_1h', 'target_direction_4h', 'target_volatility_4h']

        feature_cols = [col for col in self.df.columns if col not in exclude_cols]

        # Separar características y targets
        X = self.df[feature_cols].values
        y_regression = self.df[['target_1h', 'target_4h', 'target_24h']].values
        y_classification = self.df[['target_direction_1h', 'target_direction_4h']].values

        return X, y_regression, y_classification, feature_cols

    def create_sequences(self, X, y_reg, y_clf):
        """Crear secuencias para LSTM"""
        X_seq, y_reg_seq, y_clf_seq = [], [], []

        for i in range(self.sequence_length, len(X)):
            X_seq.append(X[i-self.sequence_length:i])
            y_reg_seq.append(y_reg[i])
            y_clf_seq.append(y_clf[i])

        return np.array(X_seq), np.array(y_reg_seq), np.array(y_clf_seq)

    def scale_features(self, X_train, X_val, X_test):
        """Escalar características de forma robusta"""
        # RobustScaler es mejor para datos financieros con outliers
        scaler = RobustScaler()
        X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1]))
        X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1]))
        X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1]))

        # Reshape back to sequences
        X_train_scaled = X_train_scaled.reshape(X_train.shape)
        X_val_scaled = X_val_scaled.reshape(X_val.shape)
        X_test_scaled = X_test_scaled.reshape(X_test.shape)

        self.scalers['feature'] = scaler
        return X_train_scaled, X_val_scaled, X_test_scaled

    def prepare_datasets(self):
        """Preparar todos los datasets"""
        X, y_reg, y_clf, feature_cols = self.prepare_features()

        # Crear secuencias
        X_seq, y_reg_seq, y_clf_seq = self.create_sequences(X, y_reg, y_clf)

        # Split temporal (importante para time series)
        split_idx1 = int(len(X_seq) * (1 - self.test_size - self.val_size))
        split_idx2 = int(len(X_seq) * (1 - self.test_size))

        X_train, X_val, X_test = (X_seq[:split_idx1],
                                 X_seq[split_idx1:split_idx2],
                                 X_seq[split_idx2:])

        y_reg_train, y_reg_val, y_reg_test = (y_reg_seq[:split_idx1],
                                            y_reg_seq[split_idx1:split_idx2],
                                            y_reg_seq[split_idx2:])

        y_clf_train, y_clf_val, y_clf_test = (y_clf_seq[:split_idx1],
                                            y_clf_seq[split_idx1:split_idx2],
                                            y_clf_seq[split_idx2:])

        # Escalar características
        X_train_scaled, X_val_scaled, X_test_scaled = self.scale_features(X_train, X_val, X_test)

        print(f"Training set: {X_train_scaled.shape}")
        print(f"Validation set: {X_val_scaled.shape}")
        print(f"Test set: {X_test_scaled.shape}")

        return (X_train_scaled, X_val_scaled, X_test_scaled,
                y_reg_train, y_reg_val, y_reg_test,
                y_clf_train, y_clf_val, y_clf_test,
                feature_cols)

# Preparar datos
preprocessor = DataPreprocessor(df_enhanced, sequence_length=60)
(X_train, X_val, X_test,
 y_reg_train, y_reg_val, y_reg_test,
 y_clf_train, y_clf_val, y_clf_test,
 feature_cols) = preprocessor.prepare_datasets()

# =============================================
# ARQUITECTURA DEL MODELO NEURONAL AVANZADO
# =============================================

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (LSTM, Dense, Input, Conv1D, MaxPooling1D,
                                   Flatten, Dropout, BatchNormalization,
                                   Attention, MultiHeadAttention, GlobalAveragePooling1D,
                                   concatenate)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ReduceLROnPlateau, EarlyStopping,
                                      ModelCheckpoint, TensorBoard)
from tensorflow.keras.regularizers import l2

class AdvancedTradingModel:
    def __init__(self, sequence_length, n_features):
        self.sequence_length = sequence_length
        self.n_features = n_features

    def build_hybrid_model(self):
        """Modelo híbrido CNN-LSTM-Attention"""
        # Input
        input_layer = Input(shape=(self.sequence_length, self.n_features))

        # CNN Branch para patrones locales
        conv1 = Conv1D(filters=64, kernel_size=3, activation='relu',
                      kernel_regularizer=l2(1e-4))(input_layer)
        conv1 = BatchNormalization()(conv1)
        conv1 = MaxPooling1D(pool_size=2)(conv1)

        conv2 = Conv1D(filters=128, kernel_size=3, activation='relu',
                      kernel_regularizer=l2(1e-4))(conv1)
        conv2 = BatchNormalization()(conv2)
        conv2 = MaxPooling1D(pool_size=2)(conv2)

        conv3 = Conv1D(filters=256, kernel_size=3, activation='relu',
                      kernel_regularizer=l2(1e-4))(conv2)
        conv3 = BatchNormalization()(conv3)

        # LSTM Branch para dependencias temporales
        lstm1 = LSTM(256, return_sequences=True,
                    kernel_regularizer=l2(1e-4))(input_layer)
        lstm1 = BatchNormalization()(lstm1)
        lstm1 = Dropout(0.3)(lstm1)

        lstm2 = LSTM(128, return_sequences=True,
                    kernel_regularizer=l2(1e-4))(lstm1)
        lstm2 = BatchNormalization()(lstm2)
        lstm2 = Dropout(0.3)(lstm2)

        # Attention mechanism
        attention = MultiHeadAttention(num_heads=8, key_dim=64)(lstm2, lstm2)
        attention = BatchNormalization()(attention)

        # Combine CNN and LSTM branches
        cnn_flat = GlobalAveragePooling1D()(conv3)
        lstm_flat = GlobalAveragePooling1D()(attention)

        # Concatenate all branches
        concatenated = concatenate([cnn_flat, lstm_flat])

        # Dense layers
        dense1 = Dense(512, activation='relu', kernel_regularizer=l2(1e-4))(concatenated)
        dense1 = BatchNormalization()(dense1)
        dense1 = Dropout(0.4)(dense1)

        dense2 = Dense(256, activation='relu', kernel_regularizer=l2(1e-4))(dense1)
        dense2 = BatchNormalization()(dense2)
        dense2 = Dropout(0.4)(dense2)

        dense3 = Dense(128, activation='relu', kernel_regularizer=l2(1e-4))(dense2)
        dense3 = BatchNormalization()(dense3)
        dense3 = Dropout(0.3)(dense3)

        # Multi-output
        # Regression outputs
        reg_output1 = Dense(1, activation='linear', name='return_1h')(dense3)
        reg_output2 = Dense(1, activation='linear', name='return_4h')(dense3)
        reg_output3 = Dense(1, activation='linear', name='return_24h')(dense3)

        # Classification outputs
        clf_output1 = Dense(1, activation='sigmoid', name='direction_1h')(dense3)
        clf_output2 = Dense(1, activation='sigmoid', name='direction_4h')(dense3)

        model = Model(inputs=input_layer,
                     outputs=[reg_output1, reg_output2, reg_output3,
                             clf_output1, clf_output2])

        return model

# Construir modelo
model_builder = AdvancedTradingModel(sequence_length=60, n_features=X_train.shape[2])
model = model_builder.build_hybrid_model()

# Compilar modelo con diferentes losses y metrics
model.compile(
    optimizer=Adam(learning_rate=0.001, clipvalue=1.0),
    loss={
        'return_1h': 'mse',
        'return_4h': 'mse',
        'return_24h': 'mse',
        'direction_1h': 'binary_crossentropy',
        'direction_4h': 'binary_crossentropy'
    },
    loss_weights={
        'return_1h': 1.0,
        'return_4h': 1.0,
        'return_24h': 0.5,
        'direction_1h': 2.0,
        'direction_4h': 1.5
    },
    metrics={
        'return_1h': ['mae', 'mse'],
        'return_4h': ['mae', 'mse'],
        'return_24h': ['mae', 'mse'],
        'direction_1h': ['accuracy', 'binary_accuracy'],
        'direction_4h': ['accuracy', 'binary_accuracy']
    }
)

model.summary()

# =============================================
# ENTRENAMIENTO DEL MODELO
# =============================================

# Callbacks avanzados
callbacks = [
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-7,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=30,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_trading_model.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    TensorBoard(
        log_dir='./logs',
        histogram_freq=1,
        update_freq='epoch'
    )
]

# SOLUCIÓN: Configurar todo el modelo dentro del strategy scope
strategy = tf.distribute.MirroredStrategy()

print(f"Number of devices: {strategy.num_replicas_in_sync}")

with strategy.scope():
    # Construir y compilar el modelo completamente dentro del scope
    model_builder = AdvancedTradingModel(sequence_length=60, n_features=X_train.shape[2])
    model = model_builder.build_hybrid_model()

    # Compilar modelo dentro del scope
    model.compile(
        optimizer=Adam(learning_rate=0.001, clipvalue=1.0),
        loss={
            'return_1h': 'mse',
            'return_4h': 'mse',
            'return_24h': 'mse',
            'direction_1h': 'binary_crossentropy',
            'direction_4h': 'binary_crossentropy'
        },
        loss_weights={
            'return_1h': 1.0,
            'return_4h': 1.0,
            'return_24h': 0.5,
            'direction_1h': 2.0,
            'direction_4h': 1.5
        },
        metrics={
            'return_1h': ['mae', 'mse'],
            'return_4h': ['mae', 'mse'],
            'return_24h': ['mae', 'mse'],
            'direction_1h': ['accuracy', 'binary_accuracy'],
            'direction_4h': ['accuracy', 'binary_accuracy']
        }
    )

print("Modelo compilado exitosamente dentro del strategy scope!")

# Ajustar batch size para distribución
batch_size = BATCH_SIZE
print(f"Batch size ajustado: {batch_size}")

# Entrenar modelo
print("Iniciando entrenamiento...")
history = model.fit(
    X_train,
    {
        'return_1h': y_reg_train[:, 0],
        'return_4h': y_reg_train[:, 1],
        'return_24h': y_reg_train[:, 2],
        'direction_1h': y_clf_train[:, 0],
        'direction_4h': y_clf_train[:, 1]
    },
    batch_size=batch_size,
    epochs=EPOCHS,
    validation_data=(
        X_val,
        {
            'return_1h': y_reg_val[:, 0],
            'return_4h': y_reg_val[:, 1],
            'return_24h': y_reg_val[:, 2],
            'direction_1h': y_clf_val[:, 0],
            'direction_4h': y_clf_val[:, 1]
        }
    ),
    callbacks=callbacks,
    verbose=1,
    shuffle=False  # Importante para time series
)

# =============================================
# EVALUACIÓN Y ANÁLISIS DEL MODELO
# =============================================

def evaluate_model_comprehensive(model, X_test, y_reg_test, y_clf_test):
    """Evaluación comprehensiva del modelo - VERSIÓN CORREGIDA"""
    # Predecir
    predictions = model.predict(X_test)

    # Evaluación de regresión
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    print("=== EVALUACIÓN DE REGRESIÓN ===")
    horizons = ['1H', '4H', '24H']

    for i, horizon in enumerate(horizons):
        # CORRECCIÓN: Asegurar que las predicciones tengan la forma correcta
        y_true_reg = y_reg_test[:, i]
        y_pred_reg = predictions[i].flatten()  # Aplanar las predicciones

        mse = mean_squared_error(y_true_reg, y_pred_reg)
        mae = mean_absolute_error(y_true_reg, y_pred_reg)
        r2 = r2_score(y_true_reg, y_pred_reg)

        print(f"Horizonte {horizon}:")
        print(f"  MSE: {mse:.6f}")
        print(f"  MAE: {mae:.6f}")
        print(f"  R²: {r2:.4f}")

    # Evaluación de clasificación
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

    print("\n=== EVALUACIÓN DE CLASIFICACIÓN ===")
    directions = ['1H', '4H']

    for i, direction in enumerate(directions):
        # CORRECCIÓN: Índices correctos para clasificación
        # Los outputs 3 y 4 son direction_1h y direction_4h
        pred_probs = predictions[i + 3].flatten()  # Aplanar probabilidades
        pred_binary = (pred_probs > 0.5).astype(int)

        y_true_clf = y_clf_test[:, i]

        acc = accuracy_score(y_true_clf, pred_binary)
        precision = precision_score(y_true_clf, pred_binary, zero_division=0)
        recall = recall_score(y_true_clf, pred_binary, zero_division=0)
        f1 = f1_score(y_true_clf, pred_binary, zero_division=0)

        print(f"Dirección {direction}:")
        print(f"  Accuracy: {acc:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  F1-Score: {f1:.4f}")

        # Matriz de confusión
        cm = confusion_matrix(y_true_clf, pred_binary)
        print(f"  Matriz de confusión:\n{cm}")

# Evaluar modelo
evaluate_model_comprehensive(model, X_test, y_reg_test, y_clf_test)

# =============================================
# BACKTESTING Y SIMULACIÓN DE TRADING
# =============================================

class TradingSimulator:
    def __init__(self, model, df, preprocessor):
        self.model = model
        self.df = df
        self.preprocessor = preprocessor

    def generate_signals(self, X_data, threshold=0.6):
        """Generar señales de trading - VERSIÓN CORREGIDA"""
        predictions = self.model.predict(X_data)

        # CORRECCIÓN: Usar el output correcto para direction_1h (índice 3)
        direction_probs = predictions[3]  # direction_1h
        signals = np.where(direction_probs > threshold, 1,
                          np.where(direction_probs < (1 - threshold), -1, 0))

        return signals.flatten(), direction_probs.flatten()

    def simulate_trading(self, initial_capital=10000, transaction_cost=0.0002):
        """Simular estrategia de trading - VERSIÓN CORREGIDA"""
        # Obtener señales para test set
        signals, probabilities = self.generate_signals(X_test)

        # CORRECCIÓN: Calcular correctamente los índices de test
        test_start_idx = len(self.df) - len(X_test) - 60  # Restar sequence_length
        prices = self.df['close'].iloc[test_start_idx:test_start_idx + len(signals)].values

        # Inicializar portfolio
        capital = initial_capital
        position = 0
        portfolio_value = []
        trades = []

        for i in range(len(signals)):
            current_price = prices[i]

            # Ejecutar señal
            if signals[i] == 1 and position <= 0:  # Señal de compra
                if position < 0:  # Cerrar short primero
                    capital += abs(position) * current_price * (1 - transaction_cost)
                    position = 0

                # Abrir long
                units = capital * 0.98 / current_price  # Usar 98% del capital
                position = units
                capital -= units * current_price * (1 + transaction_cost)
                trades.append(('BUY', current_price, probabilities[i]))

            elif signals[i] == -1 and position >= 0:  # Señal de venta
                if position > 0:  # Cerrar long primero
                    capital += position * current_price * (1 - transaction_cost)
                    position = 0

                # Abrir short
                units = capital * 0.98 / current_price
                position = -units
                capital += units * current_price * (1 - transaction_cost)
                trades.append(('SELL', current_price, probabilities[i]))

            # Calcular valor del portfolio
            portfolio_value.append(capital + position * current_price)

        # Cerrar posición final
        if position != 0:
            capital += position * prices[-1] * (1 - transaction_cost)
            portfolio_value[-1] = capital  # Actualizar último valor

        return portfolio_value, trades, capital

# Ejecutar simulación
simulator = TradingSimulator(model, df_enhanced, preprocessor)
portfolio_values, trades, final_capital = simulator.simulate_trading()

# Análisis de performance
initial_capital = 10000
total_return = (final_capital - initial_capital) / initial_capital * 100
print(f"Retorno total: {total_return:.2f}%")
print(f"Número de trades: {len(trades)}")
print(f"Capital final: ${final_capital:.2f}")

# =============================================
# VISUALIZACIONES AVANZADAS
# =============================================

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_comprehensive_visualizations(history, portfolio_values, df_enhanced, trades):
    """Crear visualizaciones comprehensivas"""

    # 1. Pérdidas del entrenamiento
    fig1 = make_subplots(rows=2, cols=2, subplot_titles=['Loss', 'Direction Accuracy', 'Returns MAE', 'Learning Rate'])

    # Loss
    fig1.add_trace(go.Scatter(y=history.history['loss'], name='Train Loss'), row=1, col=1)
    fig1.add_trace(go.Scatter(y=history.history['val_loss'], name='Val Loss'), row=1, col=1)

    # Accuracy
    fig1.add_trace(go.Scatter(y=history.history['direction_1h_accuracy'], name='Train Acc 1H'), row=1, col=2)
    fig1.add_trace(go.Scatter(y=history.history['val_direction_1h_accuracy'], name='Val Acc 1H'), row=1, col=2)

    # MAE
    fig1.add_trace(go.Scatter(y=history.history['return_1h_mae'], name='Train MAE 1H'), row=2, col=1)
    fig1.add_trace(go.Scatter(y=history.history['val_return_1h_mae'], name='Val MAE 1H'), row=2, col=1)

    fig1.update_layout(height=800, title_text="Métricas de Entrenamiento")
    fig1.show()

    # 2. Performance de trading
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(y=portfolio_values, name='Portfolio Value', line=dict(color='blue')))

    # Marcar trades
    buy_trades = [t for t in trades if t[0] == 'BUY']
    sell_trades = [t for t in trades if t[0] == 'SELL']

    if buy_trades:
        buy_indices = [portfolio_values.index(portfolio_values[-len(buy_trades):][i]) for i in range(len(buy_trades))]
        fig2.add_trace(go.Scatter(x=buy_indices, y=[portfolio_values[i] for i in buy_indices],
                             mode='markers', name='Buy', marker=dict(color='green', size=10)))

    if sell_trades:
        sell_indices = [portfolio_values.index(portfolio_values[-len(sell_trades):][i]) for i in range(len(sell_trades))]
        fig2.add_trace(go.Scatter(x=sell_indices, y=[portfolio_values[i] for i in sell_indices],
                             mode='markers', name='Sell', marker=dict(color='red', size=10)))

    fig2.update_layout(title=f"Simulación de Trading - Retorno: {total_return:.2f}%",
                      xaxis_title="Tiempo", yaxis_title="Valor del Portfolio")
    fig2.show()

# Generar visualizaciones
create_comprehensive_visualizations(history, portfolio_values, df_enhanced, trades)

# =============================================
# GUARDAR MODELO Y RECURSOS
# =============================================

# Guardar modelo completo
model.save('gbpusd_trading_model.h5')

# Guardar preprocessor
import pickle
with open('preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

# Guardar feature columns
with open('feature_columns.pkl', 'wb') as f:
    pickle.dump(feature_cols, f)

print("Modelo y recursos guardados exitosamente!")

# =============================================
# INFERENCIA EN TIEMPO REAL (EJEMPLO)
# =============================================

class RealTimePredictor:
    def __init__(self, model_path, preprocessor_path, feature_cols_path):
        self.model = tf.keras.models.load_model(model_path)
        with open(preprocessor_path, 'rb') as f:
            self.preprocessor = pickle.load(f)
        with open(feature_cols_path, 'rb') as f:
            self.feature_cols = pickle.load(f)

    def prepare_realtime_data(self, recent_data):
        """Preparar datos para predicción en tiempo real"""
        # Escalar datos
        scaled_data = self.preprocessor.scalers['feature'].transform(recent_data)
        return scaled_data.reshape(1, len(recent_data), len(self.feature_cols))

    def predict(self, recent_data):
        """Realizar predicción"""
        prepared_data = self.prepare_realtime_data(recent_data)
        predictions = self.model.predict(prepared_data)

        return {
            'return_1h': predictions[0][0][0],
            'return_4h': predictions[1][0][0],
            'return_24h': predictions[2][0][0],
            'direction_1h_prob': predictions[3][0][0],
            'direction_4h_prob': predictions[4][0][0],
            'signal_1h': 'BUY' if predictions[3][0][0] > 0.6 else 'SELL' if predictions[3][0][0] < 0.4 else 'HOLD'
        }

# Ejemplo de uso (comentado)
# predictor = RealTimePredictor('gbpusd_trading_model.h5', 'preprocessor.pkl', 'feature_columns.pkl')
# recent_data = df_enhanced[self.feature_cols].tail(60).values
# prediction = predictor.predict(recent_data)
# print("Predicción en tiempo real:", prediction)

Dataset después del feature engineering: (11561, 69)
Training set: (8050, 60, 62)
Validation set: (1150, 60, 62)
Test set: (2301, 60, 62)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 60, 62)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 58, 64)    │     11,968 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 60, 256)   │    326,656 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 58, 64)    │        256 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 256)   │      1,024 │ lstm[0][0]        │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 29, 64)    │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 60, 256)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 27, 128)   │     24,704 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 60, 128)   │    197,120 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 27, 128)   │        512 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 128)   │        512 │ lstm_1[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 13, 128)   │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 60, 128)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 11, 256)   │     98,560 │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 128)   │    263,808 │ dropout_1[0][0],  │
│ (MultiHeadAttentio… │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 11, 256)   │      1,024 │ conv1d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 128)   │        512 │ multi_head_atten… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                 

 Total params: 1,292,229 (4.93 MB)

 Trainable params: 1,288,517 (4.92 MB)

...

32/32 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - direction_1h_accuracy: 0.5487 - direction_1h_binary_accuracy: 0.5487 - direction_1h_loss: 0.6934 - direction_4h_accuracy: 0.5931 - direction_4h_binary_accuracy: 0.5931 - direction_4h_loss: 0.6659 - loss: 2.6189 - return_1h_loss: 0.0241 - return_1h_mae: 0.1133 - return_1h_mse: 0.0241 - return_24h_loss: 0.0296 - return_24h_mae: 0.1233 - return_24h_mse: 0.0296 - return_4h_loss: 0.0208 - return_4h_mae: 0.1046 - return_4h_mse: 0.0208 - val_direction_1h_accuracy: 0.4774 - val_direction_1h_binary_accuracy: 0.4774 - val_direction_1h_loss: 0.7062 - val_direction_4h_accuracy: 0.4617 - val_direction_4h_binary_accuracy: 0.4617 - val_direction_4h_loss: 0.7499 - val_loss: 2.7122 - val_return_1h_loss: 0.0013 - val_return_1h_mae: 0.0286 - val_return_1h_mse: 0.0012 - val_return_24h_loss: 0.0020 - val_return_24h_mae: 0.0410 - val_return_24h_mse: 0.0021 - val_return_4h_loss: 0.0033 - val_return_4h_mae: 0.0453 - val_return_4h_mse: 0.0029 - learning_rate: 5.0000e-04

32/32 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - direction_1h_accuracy: 0.5548 - direction_1h_binary_accuracy: 0.5548 - direction_1h_loss: 0.6895 - direction_4h_accuracy: 0.6053 - direction_4h_binary_accuracy: 0.6053 - direction_4h_loss: 0.6649 - loss: 2.5990 - return_1h_loss: 0.0194 - return_1h_mae: 0.1019 - return_1h_mse: 0.0194 - return_24h_loss: 0.0237 - return_24h_mae: 0.1114 - return_24h_mse: 0.0237 - return_4h_loss: 0.0181 - return_4h_mae: 0.0974 - return_4h_mse: 0.0181 - val_direction_1h_accuracy: 0.4791 - val_direction_1h_binary_accuracy: 0.4791 - val_direction_1h_loss: 0.7027 - val_direction_4h_accuracy: 0.4635 - val_direction_4h_binary_accuracy: 0.4635 - val_direction_4h_loss: 0.7349 - val_loss: 2.6782 - val_return_1h_loss: 6.7663e-04 - val_return_1h_mae: 0.0217 - val_return_1h_mse: 7.4436e-04 - val_return_24h_loss: 0.0012 - val_return_24h_mae: 0.0295 - val_return_24h_mse: 0.0013 - val_return_4h_loss: 0.0014 - val_return_4h_mae: 0.0287 - val_return_4h_mse: 0.0013 - learning_rate: 5.

32/32 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - direction_1h_accuracy: 0.5664 - direction_1h_binary_accuracy: 0.5664 - direction_1h_loss: 0.6843 - direction_4h_accuracy: 0.6331 - direction_4h_binary_accuracy: 0.6331 - direction_4h_loss: 0.6452 - loss: 2.5440 - return_1h_loss: 0.0134 - return_1h_mae: 0.0855 - return_1h_mse: 0.0134 - return_24h_loss: 0.0171 - return_24h_mae: 0.0941 - return_24h_mse: 0.0171 - return_4h_loss: 0.0125 - return_4h_mae: 0.0800 - return_4h_mse: 0.0125 - val_direction_1h_accuracy: 0.5061 - val_direction_1h_binary_accuracy: 0.5061 - val_direction_1h_loss: 0.6995 - val_direction_4h_accuracy: 0.4852 - val_direction_4h_binary_accuracy: 0.4852 - val_direction_4h_loss: 0.7135 - val_loss: 2.6376 - val_return_1h_loss: 6.8592e-04 - val_return_1h_mae: 0.0195 - val_return_1h_mse: 6.0719e-04 - val_return_24h_loss: 9.1878e-04 - val_return_24h_mae: 0.0259 - val_return_24h_mse: 9.8024e-04 - val_return_4h_loss: 9.1818e-04 - val_return_4h_mae: 0.0247 - val_return_4h_mse: 9.9599e-04 - l

32/32 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - direction_1h_accuracy: 0.5644 - direction_1h_binary_accuracy: 0.5644 - direction_1h_loss: 0.6843 - direction_4h_accuracy: 0.6247 - direction_4h_binary_accuracy: 0.6247 - direction_4h_loss: 0.6407 - loss: 2.5315 - return_1h_loss: 0.0120 - return_1h_mae: 0.0805 - return_1h_mse: 0.0120 - return_24h_loss: 0.0141 - return_24h_mae: 0.0864 - return_24h_mse: 0.0141 - return_4h_loss: 0.0100 - return_4h_mae: 0.0720 - return_4h_mse: 0.0100 - val_direction_1h_accuracy: 0.5078 - val_direction_1h_binary_accuracy: 0.5078 - val_direction_1h_loss: 0.6996 - val_direction_4h_accuracy: 0.4757 - val_direction_4h_binary_accuracy: 0.4757 - val_direction_4h_loss: 0.7061 - val_loss: 2.6267 - val_return_1h_loss: 6.3638e-04 - val_return_1h_mae: 0.0184 - val_return_1h_mse: 5.4749e-04 - val_return_24h_loss: 7.2525e-04 - val_return_24h_mae: 0.0215 - val_return_24h_mse: 7.2724e-04 - val_return_4h_loss: 5.3815e-04 - val_return_4h_mae: 0.0184 - val_return_4h_mse: 5.7142e-04 - l

Modelo y recursos guardados exitosamente!
